# Obtaining social science data through TDM Studio

TDM Studio is a data tool provided by [ProQuest](https://en.wikipedia.org/wiki/ProQuest), a popular social science database company that many universities subscribe to. It contains things like
- newspaper articles
- government documents
- other assorted collections

However, ProQuest wants you to use their (laggy and unreliable) Amazon Appstream-powered Jupyter instance. This notebook will show you how to download a small amount of data (30MB, ~42k processed news articles). For any larger dataset, you're stuck using their Jupyter instance.

### Login/database setup

**Step 1**: Log into TDM Studio [here](https://tdmstudio.proquest.com/home) (see the top right corner). If you're a Berkeley affiliate and don't have a TDM Studio account, click [here](http://ucblib.link/tdm-studio-request) to request one.

**Step 2**: Click on Workbench Dashboard.

**Step 3**: Select Create New Dataset and choose your database type (I chose ProQuest Databases, but you can also look through Congressional Transcripts or Newspaper Titles).

**Step 4**: Choose your databases (I chose US Dailies, a database of articles from the New York Times, Wall Street Journal, Washington Post, Los Angeles Times, and Chicago Tribune).

**Step 5**: Input parameters like keywords, date, etc. (it’s okay to leave some blank).

**Step 6**: Narrow down parameters until you're under the 2 million document limit.

**Step 7**: Once at the confirmation page, click Create Dataset.

**Step 8**: Hit Close, which will return you to the main page. Then click the On switch for your Jupyter notebook (it says it will take about 10 mins; it's often been far shorter for me).

**Step 9**: Your database needs time to finish being collected — the status will shift from In-Progress to Ready for Jupyter once it’s done.

**Step 10**: Click Open Jupyter Notebook. This button often doesn't work, so just restart your console and try again (maybe with a different browser) if it fails.

### Exporting the data

For reference, there's a notebook located at `Getting Started/2022.05.25/ProQuest TDM Studio Manuals` called `Export_Instructions.ipynb` that explains the below steps. You should have these instructions open while you read the below, as my specific AppStream instance may be different from yours.

**Step 1**: Make a new notebook.

**Step 2**: Compress your data in some way. ZIP comes preinstalled in the terminal, or you can upload and install a copy of 7zip through My Files (found in the utility row) → Temporary Files → Upload File(s).

**Step 3**: Copy and execute

 `data_to_export = 'home/ec2-user/SageMaker/YOUR_PATH_HERE'`

The landing directory from when you first start your Jupyter instance is located at `home/ec2-user/SageMaker`.

**Step 4**: Look in your Export_Instructions notebook for a line that looks like this. Copy it and execute it.

`! aws s3 cp $data_to_export s3://pq-tdm-studio-results/tdm-ale-data/YOUR_INSTITUTION_CODE_HERE/results`

**Step 5**: You will receive an email telling you whether your export was a success. If it was successful, you'll also receive a download link.

## Data cleaning

Here's some utility methods I wrote to quickly transform XML files into a DataFrame:

In [10]:
# # REQUIREMENTS (uncomment and run the below to add the requirements to your
# # requirements.txt and install them). Replace 'a' in line 8 with 'w' if you
# # want to create a new requirements.txt based on the string below.
# # Requires pip.
#
# reqs = """
# bs4==0.0.1
# pandas==2.1.1
# xmltodict==0.13.0
# """

# with open(r'requirements.txt', 'a') as reqs_txt:
#     reqs_txt.write(reqs)
#     reqs_txt.close()

# ! pip install -r requirements.txt

In [9]:
import os
import pandas as pd
from pathlib import Path
import xmltodict

def xml_to_dict(xmlpath):
    """Transform ProQuest XML file into a dictionary.
    - xml_path: String representation of the desired XML record's path
    """
    xmldict = xmltodict.parse(Path(xmlpath).read_text())['RECORD']
    collected_objects = {}

    def collect_objects(uncollected_objects, path):
        """Flatten nested dictionary into a dictionary. If a list contains
        a dictionary and non-dictionary elements, flatten the dictionary, remove
        it from the list, and leave the list unchanged otherwise. Keys are
        labeled according to their path within the nested dictionary. If
        one key would retrieve multiple values at a given path, instead assign
        this key to a list containing these values.

        Parameters:
        - uncollected_objects: dict
        - path: String representation of current path in nested dictionary
        """
        path += "/"
        for key in uncollected_objects.keys():
            if (type(uncollected_objects[key]) is not str
                        and type(uncollected_objects[key]) is list):
                to_append = []
                for subobj in uncollected_objects[key]:
                    if type(subobj) is dict:
                        collect_objects(subobj, path + str(key))
                    else:
                        to_append.append(subobj)
                collected_objects[path + key] = to_append
            elif type(uncollected_objects[key]) is not dict:
                if path + key in collected_objects.keys():
                    if type(collected_objects[path + key]) is list:
                        collected_objects[path + key].append(uncollected_objects[key])
                    else:
                        collected_objects[path + key] = [uncollected_objects[key]]
                else:
                    collected_objects[path + key] = uncollected_objects[key]

            else:
                collect_objects(uncollected_objects[key], path + str(key))

    collect_objects(xmldict, "")
    return collected_objects

def dict_to_row(xmlpath):
    """Return a DataFrame row given a String representing an XML file path.
    """
    return pd.DataFrame.from_dict(xml_to_dict(xmlpath), orient='index').T

def print_columns(df):
    """Print the columns of df in Python list format. Useful when selecting
    columns of interest in a DataFrame or to see the results of a large outer
    join.
    """
    print("[", end="")
    for i in range(len(df.columns) - 1):
        print("\"" + df.columns[i] + "\"", end=", ")
        if i % 2 == 1:
            print()
    print("\"" + df.columns[i+1] + "\"", end="")
    print("]")

ProQuest is structured such that each newspaper article in the corpus is its own XML file. You can unzip the XML files into a directory and then parse them using the below (XML → dict → DataFrame with one row → DataFrame with all rows).

In [ ]:
# Obtain empty DataFrame, which will contain all articles by the end of this
# cell.
records = pd.DataFrame()

for root, directories, filenames in os.walk('Your-data-directory'):
    # Change above string to the name of the directory your data is stored in.
    for filename in filenames:
        fpath = os.path.join(root,filename)
        if (fpath[-3:] == 'xml'):
            records = pd.concat([records, dict_to_row(fpath)],
                                ignore_index=True, join='outer')

Finally, note that document text may be in HTML. To remove the HTML, you can adapt the below code. Depending on your document format, your DataFrame may store article text in a different place (see the `print_columns` method above or `records.T` if you'd like to investigate).

In [ ]:
from bs4 import BeautifulSoup
# Strip article body of HTML
def strip_html(string):
    soup = BeautifulSoup(string, "html.parser")
    return soup.text

records['/TextInfo/Text/#text'] = records['/TextInfo/Text/#text'].apply(strip_html)